In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import numpy as np
import sklearn.metrics
from tensorflow.keras.layers import LSTM
from sklearn.metrics import confusion_matrix

In [2]:
a=pd.read_csv('Indian Liver Patient Dataset (ILPD).csv')

In [3]:
a.describe()

,age,tot_bilirubin,direct_bilirubin,tot_proteins,albumin,ag_ratio,sgpt,sgot,alkphos,is_patient
count,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,382.000000,384.000000
mean,43.460938,3.031510,1.392969,259.218750,55.505208,73.000000,6.621354,3.234115,0.964581,1.434896
std,16.343363,5.672514,2.923726,199.188744,78.473158,106.985663,1.095928,0.797259,0.326053,0.496390
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.800000,0.900000,0.300000,1.000000
25%,32.000000,0.775000,0.200000,166.750000,22.000000,24.000000,6.000000,2.700000,0.770000,1.000000
50%,44.500000,0.900000,0.300000,198.000000,31.000000,38.000000,6.750000,3.200000,1.000000,1.000000
75%,55.000000,2.000000,0.900000,275.000000,54.000000,71.250000,7.300000,3.900000,1.100000,2.000000
max,90.000000,42.800000,19.700000,1750.000000,779.000000,850.000000,9.600000,5.500000,2.800000,2.000000


In [4]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               384 non-null    int64  
 1   gender            384 non-null    object 
 2   tot_bilirubin     384 non-null    float64
 3   direct_bilirubin  384 non-null    float64
 4   tot_proteins      384 non-null    int64  
 5   albumin           384 non-null    int64  
 6   ag_ratio          384 non-null    int64  
 7   sgpt              384 non-null    float64
 8   sgot              384 non-null    float64
 9   alkphos           382 non-null    float64
 10  is_patient        384 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 33.1+ KB


In [5]:
a_1=pd.get_dummies(a['gender'])
a_1

,Female,Male
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
379,0,1
380,1,0
381,1,0
382,0,1


In [6]:
a=pd.concat([a,a_1],axis=1)

In [7]:
a.head()

,age,gender,tot_bilirubin,direct_bilirubin,tot_proteins,albumin,ag_ratio,sgpt,sgot,alkphos,is_patient,Female,Male
0,40,Male,14.5,6.4,358,50,75,5.7,2.1,0.5,1,0,1
1,26,Male,0.6,0.1,110,15,20,2.8,1.6,1.3,1,0,1
2,37,Male,0.7,0.2,235,96,54,9.5,4.9,1.0,1,0,1
3,21,Male,18.5,9.5,380,390,500,8.2,4.1,1.0,1,0,1
4,30,Male,0.7,0.2,262,15,18,9.6,4.7,1.2,1,0,1


In [8]:
x=a.drop(['gender'],axis=1)
x.head(10)

,age,tot_bilirubin,direct_bilirubin,tot_proteins,albumin,ag_ratio,sgpt,sgot,alkphos,is_patient,Female,Male
0,40,14.5,6.4,358,50,75,5.7,2.1,0.5,1,0,1
1,26,0.6,0.1,110,15,20,2.8,1.6,1.3,1,0,1
2,37,0.7,0.2,235,96,54,9.5,4.9,1.0,1,0,1
3,21,18.5,9.5,380,390,500,8.2,4.1,1.0,1,0,1
4,30,0.7,0.2,262,15,18,9.6,4.7,1.2,1,0,1
5,33,1.8,0.8,196,25,22,8.0,4.0,1.0,1,0,1
6,60,2.0,0.8,190,45,40,6.0,2.8,0.8,1,0,1
7,45,2.2,0.8,209,25,20,8.0,4.0,1.0,1,0,1
8,48,1.0,1.4,144,18,14,8.3,4.2,1.0,1,1,0
9,58,0.8,0.2,123,56,48,6.0,3.0,1.0,1,0,1


In [9]:
x.shape

(384, 12)

In [10]:
x.dtypes

age                   int64
tot_bilirubin       float64
direct_bilirubin    float64
tot_proteins          int64
albumin               int64
ag_ratio              int64
sgpt                float64
sgot                float64
alkphos             float64
is_patient            int64
Female                uint8
Male                  uint8
dtype: object

In [11]:
x=a.drop(['is_patient'],axis=1)
y=a[['is_patient']]

In [12]:
b=pd.concat([x,y],axis=1)
b

,age,gender,tot_bilirubin,direct_bilirubin,tot_proteins,albumin,ag_ratio,sgpt,sgot,alkphos,Female,Male,is_patient
0,40,Male,14.5,6.4,358,50,75,5.7,2.1,0.50,0,1,1
1,26,Male,0.6,0.1,110,15,20,2.8,1.6,1.30,0,1,1
2,37,Male,0.7,0.2,235,96,54,9.5,4.9,1.00,0,1,1
3,21,Male,18.5,9.5,380,390,500,8.2,4.1,1.00,0,1,1
4,30,Male,0.7,0.2,262,15,18,9.6,4.7,1.20,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,29,Male,1.2,0.4,160,20,22,6.2,3.0,0.90,0,1,2
380,38,Female,0.6,0.1,165,22,34,5.9,2.9,0.90,1,0,2
381,50,Female,1.0,0.3,191,22,31,7.8,4.0,1.00,1,0,2
382,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,0,1,2


In [13]:
c=b.drop(['gender'],axis=1)
c

,age,tot_bilirubin,direct_bilirubin,tot_proteins,albumin,ag_ratio,sgpt,sgot,alkphos,Female,Male,is_patient
0,40,14.5,6.4,358,50,75,5.7,2.1,0.50,0,1,1
1,26,0.6,0.1,110,15,20,2.8,1.6,1.30,0,1,1
2,37,0.7,0.2,235,96,54,9.5,4.9,1.00,0,1,1
3,21,18.5,9.5,380,390,500,8.2,4.1,1.00,0,1,1
4,30,0.7,0.2,262,15,18,9.6,4.7,1.20,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
379,29,1.2,0.4,160,20,22,6.2,3.0,0.90,0,1,2
380,38,0.6,0.1,165,22,34,5.9,2.9,0.90,1,0,2
381,50,1.0,0.3,191,22,31,7.8,4.0,1.00,1,0,2
382,60,0.5,0.1,500,20,34,5.9,1.6,0.37,0,1,2


In [14]:
c_x=c.iloc[:,:-1].values.reshape(len(c),11,1)
y=c.iloc[:,-1]


In [15]:
c_x=np.array(c_x)
c_y=np.array(y)


In [16]:
c_x.shape

(384, 11, 1)

In [17]:
x_train, x_test, y_train, y_test=train_test_split(c_x, c_y, test_size=0.40,random_state=4)
print(c_y)
print(c_x)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[[40.  ]
  [14.5 ]
  [ 6.4 ]
  ...
  [ 0.5 ]
  [ 0.  ]
  [ 1.  ]]

 [[26.  ]
  [ 0.6 ]
  [ 0.1 ]
  ...
  [ 1.3 ]
  [ 0.  ]
  [ 1.  ]]

 [[37.  ]
  [ 0.7 ]
  [ 0.2 ]
  ...
  [ 1.  ]
  [ 0.  ]
  [ 1.  ]]

 ...

 [[50.  ]


In [18]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu',input_shape=(11,1)))
model.add(MaxPool1D(pool_size=4))
model.add(LSTM(64))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])




In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 9, 64)             256       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 64)            0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 33,475
Trainable params: 33,475
Non-trainable params: 0
____________________________________________________

In [20]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
8/8 [==============================] - 12s 358ms/step - loss: 1.0212 - accuracy: 0.4391 - val_loss: 0.6926 - val_accuracy: 0.6299
Epoch 2/10
8/8 [==============================] - 0s 25ms/step - loss: 0.6821 - accuracy: 0.5913 - val_loss: 0.6402 - val_accuracy: 0.6169
Epoch 3/10
8/8 [==============================] - 0s 23ms/step - loss: 0.6424 - accuracy: 0.6478 - val_loss: 0.6269 - val_accuracy: 0.6558
Epoch 4/10
8/8 [==============================] - 0s 25ms/step - loss: 0.6491 - accuracy: 0.6261 - val_loss: 0.6277 - val_accuracy: 0.6234
Epoch 5/10
8/8 [==============================] - 0s 24ms/step - loss: 0.6426 - accuracy: 0.6478 - val_loss: 0.6263 - val_accuracy: 0.6299
Epoch 6/10
8/8 [==============================] - 0s 25ms/step - loss: 0.6288 - accuracy: 0.6304 - val_loss: 0.6284 - val_accuracy: 0.6234
Epoch 7/10
8/8 [==============================] - 0s 26ms/step - loss: 0.6283 - accuracy: 0.6391 - val_loss: 0.6251 - val_accuracy: 0.6494
Epoch 8/10
8/8 [=========

In [28]:

# predict crisp classes for test set
yhat_classes = model.predict(x_test, verbose=0)
print(yhat_classes)
pred_y=yhat_classes.argmax(axis=-1)

[[0.00676434 0.37198067 0.62125504]
 [0.00425452 0.55137753 0.44436792]
 [0.00513091 0.4838191  0.51105005]
 [0.00421523 0.6777122  0.31807262]
 [0.00437006 0.83947396 0.15615599]
 [0.00394016 0.63567674 0.360383  ]
 [0.00426324 0.60142547 0.39431125]
 [0.0037975  0.74612993 0.25007257]
 [0.0050238  0.46330455 0.53167164]
 [0.00435891 0.87141407 0.12422702]
 [0.00422337 0.5450272  0.45074943]
 [0.0041505  0.6642525  0.331597  ]
 [0.00437479 0.5863966  0.40922868]
 [0.00708742 0.3545481  0.63836455]
 [0.00448537 0.520515   0.4749996 ]
 [0.00468595 0.5473182  0.44799584]
 [0.00372186 0.72959    0.26668808]
 [0.00558036 0.47390476 0.5205149 ]
 [0.00502248 0.48080376 0.5141738 ]
 [0.0094701  0.33220536 0.65832454]
 [0.00610094 0.39687055 0.59702855]
 [0.00460256 0.57468057 0.42071682]
 [0.00366983 0.7391101  0.25722003]
 [0.00391158 0.7586581  0.2374303 ]
 [0.00413461 0.7851808  0.21068458]
 [0.00628431 0.3769005  0.61681515]
 [0.00424005 0.84251827 0.15324175]
 [0.00499988 0.4565837  0.53

In [22]:

pred_y

array([2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1,
       1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1,
       1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1,
       2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1,
       1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2],
      dtype=int64)

In [23]:
yhat_classes=yhat_classes[:,0]

In [24]:
y_test.shape

(154,)

In [25]:
tf.math.confusion_matrix(y_test,pred_y,num_classes=3, weights=None,dtype=tf.dtypes.int16,name=None)

<tf.Tensor: shape=(3, 3), dtype=int16, numpy=
array([[ 0,  0,  0],
       [ 0, 59, 27],
       [ 0, 28, 40]], dtype=int16)>

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_y))

              precision    recall  f1-score   support

           1       0.68      0.69      0.68        86
           2       0.60      0.59      0.59        68

    accuracy                           0.64       154
   macro avg       0.64      0.64      0.64       154
weighted avg       0.64      0.64      0.64       154

